In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
tf.compat.v1.disable_eager_execution()
import tensorflow_probability as tfp
import sklearn.metrics as metrics
import os
import numpy as np
import pandas as pd
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True  
sess = tf.compat.v1.Session(config=config)
set_session(sess)

import matplotlib.pyplot as plt

Using TensorFlow backend.


Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 960M, pci bus id: 0000:01:00.0, compute capability: 5.0



In [2]:
train_data_dir = 'fruits-360_dataset/fruits-360/Training'
validation_data_dir = 'fruits-360_dataset/fruits-360/Test'
test_data_dir = 'fruits-360_dataset/fruits-360/Test'


In [3]:
img_width, img_height = 100, 100

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


In [4]:
modelBayes = tf.keras.Sequential([
    tf.keras.Input(shape=input_shape),
    tfp.layers.Convolution2DReparameterization(16, kernel_size = 2,  padding="SAME", activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(pool_size=2,  padding="SAME"),
    
    tfp.layers.Convolution2DReparameterization(32, kernel_size = 2,  padding="SAME", activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(pool_size=2,  padding="SAME"),
    
    tfp.layers.Convolution2DReparameterization(64, kernel_size = 2,  padding="SAME", activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(pool_size=2,  padding="SAME"),
    
    tfp.layers.Convolution2DReparameterization(128, kernel_size = 2,  padding="SAME", activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(pool_size=2,  padding="SAME"),
    
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tfp.layers.DenseFlipout(180, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(120)
])
print(modelBayes.summary())

Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_reparameterization (C (None, 100, 100, 16)      400       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_reparameterization_1  (None, 50, 50, 32)        4128      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_reparameterization_2  (None, 25, 25, 64)        16448     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None

In [5]:
model = Sequential()
model.add(Conv2D(filters = 16, kernel_size = 2,input_shape=input_shape,padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 32,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 64,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 128,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(180))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(120,activation = 'softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 16)      208       
_________________________________________________________________
activation (Activation)      (None, 100, 100, 16)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 50, 50, 32)        2080      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 64)        8256      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 12, 12, 64)       

In [6]:

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

modelBayes.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
print('Compiled!')


Compiled!


In [7]:
batch_size = 32
epochs=30

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


Found 48431 images belonging to 120 classes.
Found 12067 images belonging to 120 classes.
Found 20622 images belonging to 120 classes.


In [8]:
checkpointer = ModelCheckpoint(filepath = 'CNN_Multiclass.hdf5', verbose = 1, save_best_only = True)

model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks = [checkpointer],
    validation_steps=validation_generator.samples // batch_size)

model.save_weights('CNN_Multiclass-Final.hdf5')


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30
 377/1513 [======>.......................] - ETA: 2:35 - loss: 0.4117 - accuracy: 0.8970
Epoch 00001: val_loss improved from inf to 0.41169, saving model to CNN_Multiclass.hdf5
1513/1513 [==============================] - 333s 220ms/step - loss: 1.6560 - accuracy: 0.5430 - val_loss: 0.4117 - val_accuracy: 0.8970
Epoch 2/30
 377/1513 [======>.......................] - ETA: 2:27 - loss: 0.3346 - accuracy: 0.9426
Epoch 00002: val_loss improved from 0.41169 to 0.33459, saving model to CNN_Multiclass.hdf5
1513/1513 [==============================] - 278s 184ms/step - loss: 0.3544 - accuracy: 0.8818 - val_loss: 0.3346 - val_accuracy: 0.9426
Epoch 3/30
 377/1513 [======>.......................] - ETA: 2:45 - loss: 0.2778 - accuracy: 0.9359
Epoch 00003: val_loss improved from 0.33459 to 0.27777, saving model to CNN_Multiclass.hdf5
1513/1513 [==============================] - 260s 172ms/step - loss: 0.2066 - 

In [9]:
checkpointer = ModelCheckpoint(filepath = 'Bayesyian_CNN_Multiclass.hdf5', verbose = 1, save_best_only = True)

modelBayes.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks = [checkpointer],
    validation_steps=validation_generator.samples // batch_size)

modelBayes.save_weights('Bayesyian_CNN_Multiclass-Final.hdf5')


Epoch 1/30
 377/1513 [======>.......................] - ETA: 1:43 - loss: 1295370.1081 - accuracy: 0.0122
Epoch 00001: val_loss improved from inf to 1295370.10809, saving model to Bayesyian_CNN_Multiclass.hdf5
1513/1513 [==============================] - 187s 123ms/step - loss: 2107995.4206 - accuracy: 0.0095 - val_loss: 1295370.1081 - val_accuracy: 0.0122
Epoch 2/30
 377/1513 [======>.......................] - ETA: 1:47 - loss: 117261.6868 - accuracy: 0.0119
Epoch 00002: val_loss improved from 1295370.10809 to 117261.68682, saving model to Bayesyian_CNN_Multiclass.hdf5
1513/1513 [==============================] - 194s 128ms/step - loss: 632516.5067 - accuracy: 0.0099 - val_loss: 117261.6868 - val_accuracy: 0.0119
Epoch 3/30
 377/1513 [======>.......................] - ETA: 1:48 - loss: 8.3118 - accuracy: 0.0114
Epoch 00003: val_loss improved from 117261.68682 to 8.31178, saving model to Bayesyian_CNN_Multiclass.hdf5
1513/1513 [==============================] - 184s 122ms/step - loss: 

C:\Users\Sebi\AppData\Roaming\Python\Python36\site-packages\tensorflow_core\python\keras\callbacks.py:1020: RuntimeWarning: invalid value encountered in less
  if self.monitor_op(current, self.best):


 377/1513 [======>.......................] - ETA: 1:50 - loss: nan - accuracy: 0.0081- ETA:
Epoch 00018: val_loss did not improve from 8.20651
1513/1513 [==============================] - 186s 123ms/step - loss: nan - accuracy: 0.0081 - val_loss: nan - val_accuracy: 0.0081
Epoch 19/30
 377/1513 [======>.......................] - ETA: 1:41 - loss: nan - accuracy: 0.0081
Epoch 00019: val_loss did not improve from 8.20651
1513/1513 [==============================] - 184s 121ms/step - loss: nan - accuracy: 0.0081 - val_loss: nan - val_accuracy: 0.0081
Epoch 20/30
 377/1513 [======>.......................] - ETA: 1:46 - loss: nan - accuracy: 0.0081
Epoch 00020: val_loss did not improve from 8.20651
1513/1513 [==============================] - 177s 117ms/step - loss: nan - accuracy: 0.0081 - val_loss: nan - val_accuracy: 0.0081
Epoch 21/30
 377/1513 [======>.......................] - ETA: 1:43 - loss: nan - accuracy: 0.0081
Epoch 00021: val_loss did not improve from 8.20651
1513/1513 [======

In [10]:
# evaluate the model
test_steps_per_epoch = np.math.ceil(test_generator.samples / test_generator.batch_size)
scores = model.evaluate_generator(test_generator)
predictions = model.predict_generator(test_generator, steps=test_steps_per_epoch)
# Get most likely class
predicted_classes = np.argmax(predictions, axis=1)

true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())  


report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report) 


Instructions for updating:
Please use Model.evaluate, which supports generators.
Instructions for updating:
Please use Model.predict, which supports generators.
                     precision    recall  f1-score   support

     Apple Braeburn       0.01      0.01      0.01       164
 Apple Crimson Snow       0.01      0.01      0.01       148
     Apple Golden 1       0.01      0.01      0.01       164
     Apple Golden 2       0.01      0.01      0.01       164
     Apple Golden 3       0.01      0.01      0.01       161
 Apple Granny Smith       0.01      0.01      0.01       164
    Apple Pink Lady       0.01      0.01      0.01       152
        Apple Red 1       0.00      0.00      0.00       164
        Apple Red 2       0.01      0.01      0.01       164
        Apple Red 3       0.01      0.01      0.01       144
Apple Red Delicious       0.00      0.00      0.00       166
 Apple Red Yellow 1       0.01      0.01      0.01       164
 Apple Red Yellow 2       0.02      0.02     

In [11]:
# evaluate the modelBayes
test_steps_per_epoch = np.math.ceil(test_generator.samples / test_generator.batch_size)
scores = modelBayes.evaluate_generator(test_generator)
predictions = modelBayes.predict_generator(test_generator, steps=test_steps_per_epoch)
# Get most likely class
predicted_classes = np.argmax(predictions, axis=1)

true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())  


report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report) 

                     precision    recall  f1-score   support

     Apple Braeburn       0.01      1.00      0.02       164
 Apple Crimson Snow       0.00      0.00      0.00       148
     Apple Golden 1       0.00      0.00      0.00       164
     Apple Golden 2       0.00      0.00      0.00       164
     Apple Golden 3       0.00      0.00      0.00       161
 Apple Granny Smith       0.00      0.00      0.00       164
    Apple Pink Lady       0.00      0.00      0.00       152
        Apple Red 1       0.00      0.00      0.00       164
        Apple Red 2       0.00      0.00      0.00       164
        Apple Red 3       0.00      0.00      0.00       144
Apple Red Delicious       0.00      0.00      0.00       166
 Apple Red Yellow 1       0.00      0.00      0.00       164
 Apple Red Yellow 2       0.00      0.00      0.00       219
            Apricot       0.00      0.00      0.00       164
            Avocado       0.00      0.00      0.00       143
       Avocado ripe    

C:\Users\Sebi\.conda\envs\keras-gpu\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
